# Signal Mapping Fundamentals

## Purpose

This notebook teaches you how to map signals from point cloud data to voxel grids using various interpolation methods. You'll learn to use nearest neighbor, linear, IDW, and KDE interpolation with interactive widgets.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Understand signal mapping concept and importance
- ✅ Use all interpolation methods interactively
- ✅ Compare method performance and accuracy in real-time
- ✅ Select appropriate method for your use case
- ✅ Understand parameter effects on mapping quality

## Estimated Duration

60-90 minutes

---

## Overview

Signal mapping is the process of interpolating point cloud data (with signals) onto a regular voxel grid. The AM-QADF framework supports multiple interpolation methods:

- 🎯 **Nearest Neighbor**: Fast, assigns each voxel to its nearest point
- 📈 **Linear**: Smooth interpolation using k-nearest neighbors
- ⚖️ **IDW (Inverse Distance Weighting)**: Distance-weighted interpolation
- 📊 **Gaussian KDE**: Kernel density estimation for smooth surfaces

Use the interactive widgets below to explore signal mapping - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Optional, Tuple, Dict, Any

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import infrastructure
INFRASTRUCTURE_AVAILABLE = False
try:
    from src.infrastructure.database import get_connection_manager
    INFRASTRUCTURE_AVAILABLE = True
except (ImportError, TypeError, Exception) as e:
    INFRASTRUCTURE_AVAILABLE = False
    print(f"⚠️ Infrastructure layer not available: {type(e).__name__}: {e}")

# Try to import signal mapping classes
SIGNAL_MAPPING_AVAILABLE = False
try:
    from am_qadf.signal_mapping.execution.sequential import interpolate_to_voxels
    from am_qadf.signal_mapping.methods.nearest_neighbor import NearestNeighborInterpolation
    from am_qadf.signal_mapping.methods.linear import LinearInterpolation
    from am_qadf.signal_mapping.methods.idw import IDWInterpolation
    from am_qadf.signal_mapping.methods.kde import GaussianKDEInterpolation
    SIGNAL_MAPPING_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ Signal mapping classes not available: {e} - using demo mode")

# Try to import VoxelGrid
VOXEL_AVAILABLE = False
try:
    from am_qadf.voxelization.voxel_grid import VoxelGrid
    VOXEL_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ VoxelGrid not available: {e} - using demo mode")

# Try to import query clients for loading data
QUERY_CLIENTS_AVAILABLE = False
mongo_client = None
unified_client = None
voxel_storage = None
alignment_storage = None

# Try to import alignment storage
ALIGNMENT_STORAGE_AVAILABLE = False
try:
    from am_qadf.synchronization import AlignmentStorage
    ALIGNMENT_STORAGE_AVAILABLE = True
except ImportError as e:
    ALIGNMENT_STORAGE_AVAILABLE = False
    print(f"⚠️ AlignmentStorage not available: {e}")

if INFRASTRUCTURE_AVAILABLE:
    try:
        manager = get_connection_manager(env_name="development")
        mongo_client = manager.get_mongodb_client()
        
        if mongo_client and mongo_client.is_connected():
            from am_qadf.query import UnifiedQueryClient
            unified_client = UnifiedQueryClient(mongo_client=mongo_client)
            QUERY_CLIENTS_AVAILABLE = True
            print("✅ MongoDB connection established")
            
            # Initialize alignment storage if available
            if ALIGNMENT_STORAGE_AVAILABLE:
                try:
                    alignment_storage = AlignmentStorage(mongo_client=mongo_client)
                    print("✅ AlignmentStorage initialized")
                except Exception as e:
                    print(f"⚠️ AlignmentStorage initialization failed: {e}")
            
            # Try to import voxel grid storage
            try:
                import importlib.util
                voxel_storage_path = src_dir / 'am_qadf' / 'voxel_domain' / 'voxel_storage.py'
                if voxel_storage_path.exists():
                    spec = importlib.util.spec_from_file_location("voxel_storage", voxel_storage_path)
                    voxel_storage_module = importlib.util.module_from_spec(spec)
                    spec.loader.exec_module(voxel_storage_module)
                    VoxelGridStorage = voxel_storage_module.VoxelGridStorage
                    voxel_storage = VoxelGridStorage(mongo_client)
                    print("✅ VoxelGridStorage initialized")
            except Exception as e:
                print(f"⚠️ VoxelGridStorage not available: {e}")
    except Exception as e:
        print(f"⚠️ MongoDB connection failed: {type(e).__name__}: {e}")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env


Failed to connect to MongoDB: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 6960106e9eb3f75b293ce527, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


Failed to initialize MongoDB connection: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 6960106e9eb3f75b293ce527, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


✅ Setup complete!


## Interactive Signal Mapping Interface

Use the widgets below to map signals to voxel grids. Select interpolation method, configure parameters, and visualize results interactively!


In [2]:
# Create Interactive Signal Mapping Interface

# Global state
current_points = None
current_signals = None
current_grid = None
mapped_grid = None
sample_data_generated = False
current_model_id = None
current_grid_id = None
loaded_grid_data = None

# ============================================
# Helper Functions for Demo Data
# ============================================

def generate_sample_data(n_points=1000, bbox_min=(-50, -50, 0), bbox_max=(50, 50, 100)):
    """Generate sample point cloud data with signals."""
    np.random.seed(42)
    
    # Generate random points in bounding box
    points = np.random.uniform(
        low=bbox_min,
        high=bbox_max,
        size=(n_points, 3)
    )
    
    # Generate signals
    # Signal 1: Temperature (smooth gradient)
    x_norm = (points[:, 0] - bbox_min[0]) / (bbox_max[0] - bbox_min[0])
    y_norm = (points[:, 1] - bbox_min[1]) / (bbox_max[1] - bbox_min[1])
    z_norm = (points[:, 2] - bbox_min[2]) / (bbox_max[2] - bbox_min[2])
    
    temperature = 500 + 300 * np.sin(2 * np.pi * x_norm) * np.cos(2 * np.pi * y_norm) + 200 * z_norm
    temperature += np.random.normal(0, 10, n_points)  # Add noise
    
    # Signal 2: Power (with hotspots)
    power = 200 + 100 * np.exp(-((points[:, 0] - 10)**2 + (points[:, 1] - 10)**2) / 100)
    power += 50 * np.exp(-((points[:, 0] + 20)**2 + (points[:, 1] + 20)**2) / 200)
    power += np.random.normal(0, 5, n_points)
    
    # Signal 3: Density (layered pattern)
    density = 0.8 + 0.2 * np.sin(2 * np.pi * z_norm * 5) + np.random.normal(0, 0.02, n_points)
    
    signals = {
        'temperature': temperature,
        'power': power,
        'density': density
    }
    
    return points, signals

# ============================================
# Top Panel: Method Selection and Actions
# ============================================

method_label = widgets.HTML("<b>Interpolation Method:</b>")
method_selector = RadioButtons(
    options=[
        ('Nearest Neighbor', 'nearest'),
        ('Linear', 'linear'),
        ('IDW (Inverse Distance Weighting)', 'idw'),
        ('Gaussian KDE', 'gaussian_kde')
    ],
    value='nearest',
    description='Method:',
    style={'description_width': 'initial'}
)

# Data Source Selection
data_source_label = widgets.HTML("<b>Data Source:</b>")
data_source_mode = RadioButtons(
    options=[('From MongoDB', 'mongodb'), ('Generate Sample', 'sample')],
    value='mongodb',
    description='Source:',
    style={'description_width': 'initial'}
)

# Model and Grid Selection (for MongoDB)
model_label = widgets.HTML("<b>Model & Grid:</b>")
models = []
model_options = [("━━━ Select Model ━━━", None)]

if unified_client and mongo_client:
    try:
        from am_qadf.query import STLModelClient
        stl_client = STLModelClient(mongo_client=mongo_client)
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", m.get('model_id'))
            for m in models
        ])
        if len(model_options) == 1:
            model_options.append(("No models available", None))
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")
        model_options.append(("Error loading models", None))

model_dropdown = Dropdown(
    options=model_options,
    value=model_options[1][1] if len(model_options) > 1 else None,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='300px', display='flex')
)

# Grid selection (populated when model is selected)
grid_options = [("━━━ Select Grid ━━━", None)]
grid_dropdown = Dropdown(
    options=grid_options,
    value=None,
    description='Grid:',
    style={'description_width': 'initial'},
    layout=Layout(width='300px', display='none')
)

# Signal data source selection
signal_source_label = widgets.HTML("<b>Signal Data:</b>")
signal_source_selector = RadioButtons(
    options=[
        ('Laser Parameters', 'laser'),
        ('ISPM Monitoring', 'ispm'),
        ('Hatching Paths', 'hatching'),
        ('CT Scan', 'ct'),
        ('All Sources', 'all')
    ],
    value='all',  # Default to 'all' to query all sources by default
    description='Source:',
    style={'description_width': 'initial'}
)

load_data_button = Button(
    description='Load from MongoDB',
    button_style='info',
    icon='database',
    layout=Layout(width='180px')
)

generate_data_button = Button(
    description='Generate Sample Data',
    button_style='',
    icon='refresh',
    layout=Layout(width='180px')
)

map_button = Button(
    description='Map All Signals',
    button_style='success',
    icon='map',
    layout=Layout(width='150px')
)

save_mapped_button = Button(
    description='Save Mapped Grid',
    button_style='info',
    icon='save',
    layout=Layout(width='150px', display='none')
)

load_mapped_button = Button(
    description='Load Mapped Grid',
    button_style='',
    icon='folder-open',
    layout=Layout(width='150px')
)

compare_button = Button(
    description='Compare Methods',
    button_style='',
    icon='copy',
    layout=Layout(width='150px')
)

def update_data_source_ui(change):
    """Show/hide UI based on data source mode."""
    if change['new'] == 'mongodb':
        model_dropdown.layout.display = 'flex'
        grid_dropdown.layout.display = 'flex'
        signal_source_selector.layout.display = 'flex'
        load_data_button.layout.display = 'flex'
        generate_data_button.layout.display = 'none'
    else:
        model_dropdown.layout.display = 'none'
        grid_dropdown.layout.display = 'none'
        signal_source_selector.layout.display = 'none'
        load_data_button.layout.display = 'none'
        generate_data_button.layout.display = 'flex'

data_source_mode.observe(update_data_source_ui, names='value')
update_data_source_ui({'new': data_source_mode.value})

top_panel = VBox([
    HBox([
        method_label,
        method_selector,
        data_source_label,
        data_source_mode
    ], layout=Layout(justify_content='flex-start', padding='5px')),
    HBox([
        model_label,
        model_dropdown,
        grid_dropdown,
        signal_source_label,
        signal_source_selector
    ], layout=Layout(justify_content='flex-start', padding='5px')),
    HBox([
        load_data_button,
        generate_data_button,
        map_button,
        save_mapped_button,
        load_mapped_button,
        compare_button
    ], layout=Layout(justify_content='flex-start', padding='5px'))
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Method Parameters
# ============================================

# Method-specific parameter sections
params_label = widgets.HTML("<b>Method Parameters:</b>")

# Nearest Neighbor parameters (none needed, but show info)
nn_info = widgets.HTML("<p>No parameters required. Fastest method.</p>")
nn_params = VBox([nn_info], layout=Layout(display='flex'))

# Linear parameters
linear_k_neighbors = IntSlider(value=8, min=1, max=50, step=1, description='K Neighbors:', style={'description_width': 'initial'})
linear_radius = FloatSlider(value=10.0, min=0.1, max=100.0, step=0.1, description='Radius (mm):', style={'description_width': 'initial'})
linear_use_radius = Checkbox(value=False, description='Use Radius Limit', style={'description_width': 'initial'})
linear_params = VBox([
    linear_k_neighbors,
    linear_use_radius,
    linear_radius
], layout=Layout(display='none'))

# IDW parameters
idw_power = FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Power:', style={'description_width': 'initial'})
idw_k_neighbors = IntSlider(value=8, min=1, max=50, step=1, description='K Neighbors:', style={'description_width': 'initial'})
idw_radius = FloatSlider(value=10.0, min=0.1, max=100.0, step=0.1, description='Radius (mm):', style={'description_width': 'initial'})
idw_use_radius = Checkbox(value=False, description='Use Radius Limit', style={'description_width': 'initial'})
idw_params = VBox([
    idw_power,
    idw_k_neighbors,
    idw_use_radius,
    idw_radius
], layout=Layout(display='none'))

# KDE parameters
kde_bandwidth = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Bandwidth:', style={'description_width': 'initial'})
kde_adaptive = Checkbox(value=False, description='Adaptive Bandwidth', style={'description_width': 'initial'})
kde_kernel = Dropdown(
    options=[('Gaussian', 'gaussian'), ('Epanechnikov', 'epanechnikov'), ('Tophat', 'tophat')],
    value='gaussian',
    description='Kernel:',
    style={'description_width': 'initial'}
)
kde_params = VBox([
    kde_bandwidth,
    kde_adaptive,
    kde_kernel
], layout=Layout(display='none'))

def update_method_params(change):
    """Show/hide parameter controls based on selected method."""
    method = change['new']
    nn_params.layout.display = 'none'
    linear_params.layout.display = 'none'
    idw_params.layout.display = 'none'
    kde_params.layout.display = 'none'
    
    if method == 'nearest':
        nn_params.layout.display = 'flex'
    elif method == 'linear':
        linear_params.layout.display = 'flex'
    elif method == 'idw':
        idw_params.layout.display = 'flex'
    elif method == 'gaussian_kde':
        kde_params.layout.display = 'flex'

method_selector.observe(update_method_params, names='value')
update_method_params({'new': method_selector.value})  # Initialize

# Grid Resolution Section
resolution_label = widgets.HTML("<b>Grid Resolution:</b>")
grid_resolution = FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Resolution (mm):', style={'description_width': 'initial'})

# Signal Selector (for visualization after mapping)
signal_label = widgets.HTML("<b>Signal to Visualize:</b>")
signal_selector = Dropdown(
    options=[('Temperature', 'temperature'), ('Power', 'power'), ('Density', 'density')],
    value='temperature',
    description='Signal:',
    style={'description_width': 'initial'}
)

# Data Info
data_info_label = widgets.HTML("<b>Data Info:</b>")
data_info = widgets.HTML("No data generated yet")

left_panel = VBox([
    params_label,
    nn_params,
    linear_params,
    idw_params,
    kde_params,
    resolution_label,
    grid_resolution,
    signal_label,
    signal_selector,
    data_info_label,
    data_info
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('2D Slice', '2d'), ('3D View', '3d'), ('Comparison', 'compare')],
    value='2d',
    description='View:',
    style={'description_width': 'initial'}
)

# 2D Slice visualization
slice_axis = RadioButtons(
    options=[('XY', 'xy'), ('XZ', 'xz'), ('YZ', 'yz')],
    value='xy',
    description='Slice Axis:',
    style={'description_width': 'initial'}
)

slice_position = IntSlider(value=50, min=0, max=100, step=1, description='Position (%):', style={'description_width': 'initial'})

viz_output = Output(layout=Layout(height='500px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Visualization</h3>"),
    viz_mode,
    slice_axis,
    slice_position,
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Results and Metrics
# ============================================

# Mapping Results
results_label = widgets.HTML("<b>Mapping Results:</b>")
results_display = widgets.HTML("No mapping performed yet")
results_section = VBox([
    results_label,
    results_display
], layout=Layout(padding='5px'))

# Performance Metrics
metrics_label = widgets.HTML("<b>Performance Metrics:</b>")
metrics_display = widgets.HTML("No metrics available")
metrics_section = VBox([
    metrics_label,
    metrics_display
], layout=Layout(padding='5px'))

# Method Comparison
comparison_label = widgets.HTML("<b>Method Comparison:</b>")
comparison_display = widgets.HTML("No comparison available")
comparison_section = VBox([
    comparison_label,
    comparison_display
], layout=Layout(padding='5px'))

right_panel = VBox([
    results_section,
    metrics_section,
    comparison_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

status_display = widgets.HTML("<b>Status:</b> Ready to map signals")
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)
error_display = widgets.HTML("")

bottom_panel = VBox([
    status_display,
    progress_bar,
    error_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Helper Functions
# ============================================

def update_grid_dropdown(change):
    """Update grid dropdown when model is selected."""
    global grid_options
    
    model_id = change['new']
    grid_options = [("━━━ Select Grid ━━━", None)]
    
    if model_id and voxel_storage:
        try:
            available_grids = voxel_storage.list_grids(model_id=model_id, limit=100)
            grid_options.extend([
                (f"{g.get('grid_name', 'Unknown')} ({g.get('grid_id', '')[:8]}...)", g.get('grid_id'))
                for g in available_grids
            ])
            if len(grid_options) == 1:
                grid_options.append(("No grids available", None))
        except Exception as e:
            print(f"⚠️ Error loading grids: {e}")
            grid_options.append(("Error loading grids", None))
    
    grid_dropdown.options = grid_options
    if len(grid_options) > 1:
        grid_dropdown.value = grid_options[1][1] if grid_options[1][1] else None

_loading_in_progress = False

def auto_load_data(change):
    """Auto-load data when both model and grid are selected."""
    model_id = model_dropdown.value
    grid_id = grid_dropdown.value
    
    # Only auto-load if both are selected and we're in MongoDB mode
    if data_source_mode.value == 'mongodb' and model_id and grid_id:
        load_from_mongodb(None)

model_dropdown.observe(update_grid_dropdown, names='value')
grid_dropdown.observe(auto_load_data, names='value')

def load_from_mongodb(button=None):
    """Load grid and signal data from MongoDB. Can be called manually or auto-triggered."""
    global current_points, current_signals, current_grid, current_model_id, current_grid_id, loaded_grid_data, sample_data_generated, _loading_in_progress
    
    # Prevent multiple simultaneous loads
    if _loading_in_progress:
        return
    
    if not unified_client or not voxel_storage or not mongo_client:
        error_display.value = "<span style='color: red;'>❌ MongoDB not available. Cannot load data.</span>"
        status_display.value = "<b>Status:</b> <span style='color: red;'>Error: MongoDB unavailable</span>"
        return
    
    _loading_in_progress = True
    
    model_id = model_dropdown.value
    grid_id = grid_dropdown.value
    
    if not model_id:
        error_display.value = "<span style='color: red;'>⚠️ Please select a model from the dropdown</span>"
        status_display.value = "<b>Status:</b> <span style='color: red;'>No model selected</span>"
        return
    
    if not grid_id:
        error_display.value = "<span style='color: red;'>⚠️ Please select a grid from the dropdown</span>"
        status_display.value = "<b>Status:</b> <span style='color: red;'>No grid selected</span>"
        return
    
    status_display.value = "<b>Status:</b> Loading data from MongoDB..."
    progress_bar.value = 0
    error_display.value = ""
    
    try:
        current_model_id = model_id
        current_grid_id = grid_id
        
        # Load grid
        progress_bar.value = 20
        status_display.value = "<b>Status:</b> Loading grid from MongoDB..."
        loaded_grid_data = voxel_storage.load_voxel_grid(grid_id)
        
        if not loaded_grid_data:
            error_display.value = f"<span style='color: red;'>❌ Failed to load grid {grid_id[:8]}...</span>"
            status_display.value = "<b>Status:</b> <span style='color: red;'>Error loading grid</span>"
            progress_bar.value = 0
            return
        
        # Reconstruct grid from metadata
        metadata = loaded_grid_data.get('metadata', {})
        bbox_min = tuple(metadata.get('bbox_min', [-50, -50, 0]))
        bbox_max = tuple(metadata.get('bbox_max', [50, 50, 100]))
        resolution = metadata.get('resolution', 2.0)
        
        if VOXEL_AVAILABLE:
            current_grid = VoxelGrid(
                bbox_min=bbox_min,
                bbox_max=bbox_max,
                resolution=resolution,
                aggregation=metadata.get('aggregation', 'mean')
            )
        else:
            # Demo grid
            class DemoGrid:
                def __init__(self, bbox_min, bbox_max, resolution):
                    self.bbox_min = np.array(bbox_min)
                    self.bbox_max = np.array(bbox_max)
                    self.resolution = resolution
                    self.size = self.bbox_max - self.bbox_min
                    self.dims = np.ceil(self.size / resolution).astype(int)
                    self.dims = np.maximum(self.dims, [1, 1, 1])
                    self.voxels = {}
                    self.available_signals = set()
            current_grid = DemoGrid(bbox_min, bbox_max, resolution)
        
        progress_bar.value = 50
        
        # Check if grid was created from alignment
        alignment_id = metadata.get('alignment_id')
        created_from_alignment = metadata.get('created_from_alignment', False)
        use_aligned_data = created_from_alignment and alignment_id and alignment_storage
        
        # Load signal data
        if use_aligned_data:
            status_display.value = "<b>Status:</b> Loading aligned data from alignment storage..."
            progress_bar.value = 60
        else:
            status_display.value = "<b>Status:</b> Loading signal data from MongoDB..."
            progress_bar.value = 60
        
        signal_source = signal_source_selector.value
        
        # Try to load aligned data first if available
        aligned_data_loaded = False
        if use_aligned_data:
            try:
                alignment = alignment_storage.load_alignment(alignment_id, load_aligned_data=True)
                if alignment and 'aligned_data' in alignment:
                    aligned_data = alignment['aligned_data']
                    aligned_data_loaded = True
                    status_display.value = "<b>Status:</b> ✅ Using aligned data for signal mapping"
            except Exception as e:
                error_display.value = f"<span style='color: orange;'>⚠️ Failed to load aligned data: {e}. Falling back to raw query data.</span>"
                aligned_data_loaded = False
        
        # Query data from MongoDB (fallback or if no aligned data)
        if not aligned_data_loaded:
            all_data = unified_client.get_all_data(model_id)
        
        points_list = []
        signals_dict = {}
        
        # Helper function to extract points and signals from QueryResult
        def extract_data(data, source_name):
            if data is None:
                return
            if hasattr(data, 'points') and data.points is not None:
                points = data.points
                # Convert to numpy array if it's a list
                if isinstance(points, list):
                    points = np.array(points)
                elif not isinstance(points, np.ndarray):
                    points = np.array(points)
                
                # Only add if we have valid points
                if len(points) > 0:
                    points_list.append(points)
                    
                    if hasattr(data, 'signals') and data.signals:
                        for signal_name, signal_values in data.signals.items():
                            # Convert signal values to numpy array if needed
                            if isinstance(signal_values, list):
                                signal_values = np.array(signal_values)
                            elif not isinstance(signal_values, np.ndarray):
                                signal_values = np.array(signal_values)
                            
                            signal_key = f"{source_name}_{signal_name}" if source_name else signal_name
                            if signal_key not in signals_dict:
                                signals_dict[signal_key] = []
                            signals_dict[signal_key].append(signal_values)
        
        # Helper function to extract points and signals from aligned data
        def extract_aligned_data(source_data, source_name):
            if not isinstance(source_data, dict):
                return
            if 'points' in source_data and source_data['points'] is not None:
                points = source_data['points']
                # Convert to numpy array if needed
                if isinstance(points, list):
                    points = np.array(points)
                elif not isinstance(points, np.ndarray):
                    points = np.array(points)
                
                # Only add if we have valid points
                if len(points) > 0:
                    points_list.append(points)
                    
                    # Extract signals if available
                    if 'signals' in source_data and source_data['signals'] is not None:
                        signals = source_data['signals']
                        
                        # Handle both dict (multiple signals) and array (single signal) formats
                        if isinstance(signals, dict):
                            # Multiple signals stored as dict
                            for signal_name, signal_values in signals.items():
                                # Convert signal values to numpy array if needed
                                if isinstance(signal_values, list):
                                    signal_values = np.array(signal_values)
                                elif not isinstance(signal_values, np.ndarray):
                                    signal_values = np.array(signal_values)
                                
                                # Only add if signal has valid length
                                if len(signal_values) > 0:
                                    signal_key = f"{source_name}_{signal_name}" if source_name else signal_name
                                    if signal_key not in signals_dict:
                                        signals_dict[signal_key] = []
                                    signals_dict[signal_key].append(signal_values)
                        elif isinstance(signals, np.ndarray):
                            # Single signal stored as array - use source name as signal name
                            if len(signals) > 0:
                                signal_key = f"{source_name}_signal" if source_name else "signal"
                                if signal_key not in signals_dict:
                                    signals_dict[signal_key] = []
                                signals_dict[signal_key].append(signals)
                        elif isinstance(signals, list):
                            # Convert list to numpy array
                            signals = np.array(signals)
                            if len(signals) > 0:
                                signal_key = f"{source_name}_signal" if source_name else "signal"
                                if signal_key not in signals_dict:
                                    signals_dict[signal_key] = []
                                signals_dict[signal_key].append(signals)
        
        # Load data based on selected source
        if aligned_data_loaded:
            # Use aligned data
            if signal_source in ['laser', 'all'] and 'laser' in aligned_data:
                extract_aligned_data(aligned_data['laser'], 'laser')
            
            if signal_source in ['ispm', 'all'] and 'ispm' in aligned_data:
                extract_aligned_data(aligned_data['ispm'], 'ispm')
            
            if signal_source in ['hatching', 'all'] and 'hatching' in aligned_data:
                extract_aligned_data(aligned_data['hatching'], 'hatching')
            
            if signal_source in ['ct', 'all'] and 'ct' in aligned_data:
                extract_aligned_data(aligned_data['ct'], 'ct')
        else:
            # Use raw query data (fallback)
            if signal_source in ['laser', 'all']:
                extract_data(all_data.get('laser_parameters'), 'laser')
            
            if signal_source in ['ispm', 'all']:
                extract_data(all_data.get('ispm_monitoring'), 'ispm')
            
            if signal_source in ['hatching', 'all']:
                extract_data(all_data.get('hatching_layers'), 'hatching')
            
            if signal_source in ['ct', 'all']:
                extract_data(all_data.get('ct_scan'), 'ct')
        
        if not points_list:
            if aligned_data_loaded:
                error_display.value = "<span style='color: orange;'>⚠️ No aligned data found for selected source. Using sample data.</span>"
            else:
                error_display.value = "<span style='color: orange;'>⚠️ No signal data found for selected source. Using sample data.</span>"
            # Fallback to sample data
            bbox_min_arr = np.array(bbox_min)
            bbox_max_arr = np.array(bbox_max)
            current_points, current_signals = generate_sample_data(
                n_points=1000,
                bbox_min=tuple(bbox_min_arr),
                bbox_max=tuple(bbox_max_arr)
            )
        else:
            # Convert all points to numpy arrays first
            points_arrays = []
            for p in points_list:
                if isinstance(p, list):
                    p = np.array(p)
                elif not isinstance(p, np.ndarray):
                    p = np.array(p)
                points_arrays.append(p)
            
            # Combine points (handle different shapes)
            if len(points_arrays) > 1:
                # Check if all have same shape
                shapes = [p.shape for p in points_arrays]
                if all(s == shapes[0] for s in shapes):
                    current_points = np.vstack(points_arrays)
                else:
                    # Concatenate if different shapes
                    current_points = np.concatenate(points_arrays, axis=0)
            else:
                current_points = points_arrays[0]
            
            # Ensure points is numpy array and 2D (N, 3)
            if not isinstance(current_points, np.ndarray):
                current_points = np.array(current_points)
            
            if current_points.ndim == 1:
                current_points = current_points.reshape(-1, 3)
            elif current_points.ndim == 2 and current_points.shape[1] != 3:
                # Try to reshape if wrong shape
                if current_points.size % 3 == 0:
                    current_points = current_points.reshape(-1, 3)
                else:
                    error_display.value = f"<span style='color: red;'>❌ Points array has incompatible shape: {current_points.shape}</span>"
                    return
            
            # Combine signals
            current_signals = {}
            for signal_name, signal_arrays in signals_dict.items():
                # Convert all to numpy arrays
                arrays = []
                for arr in signal_arrays:
                    if isinstance(arr, list):
                        arr = np.array(arr)
                    elif not isinstance(arr, np.ndarray):
                        arr = np.array(arr)
                    arrays.append(arr)
                
                if len(arrays) > 1:
                    # Concatenate arrays
                    try:
                        current_signals[signal_name] = np.concatenate(arrays)
                    except:
                        # If shapes don't match, try to flatten and concatenate
                        flattened = []
                        for arr in arrays:
                            if isinstance(arr, np.ndarray):
                                if arr.ndim > 1:
                                    flattened.append(arr.flatten())
                                else:
                                    flattened.append(arr)
                            else:
                                flattened.append(np.array(arr).flatten())
                        current_signals[signal_name] = np.concatenate(flattened)
                else:
                    current_signals[signal_name] = arrays[0] if arrays else np.array([])
            
            # Ensure signal arrays match point count
            n_points = len(current_points)
            for signal_name in list(current_signals.keys()):
                signal_array = current_signals[signal_name]
                if len(signal_array) != n_points:
                    # Truncate or pad to match
                    if len(signal_array) > n_points:
                        current_signals[signal_name] = signal_array[:n_points]
                    else:
                        # Pad with last value
                        padding = np.full(n_points - len(signal_array), signal_array[-1] if len(signal_array) > 0 else 0.0)
                        current_signals[signal_name] = np.concatenate([signal_array, padding])
            
            # Check if signals were extracted
            if not current_signals:
                # If no signals found but we have points
                if aligned_data_loaded:
                    # Aligned data should have signals - if not, user needs to re-run alignment
                    error_display.value = """
                    <span style='color: red;'>
                    <b>❌ Aligned data has points but no signals.</b><br>
                    <b>Action Required:</b> Please go to Notebook 02 (Temporal and Spatial Alignment) and:<br>
                    1. Re-run the alignment<br>
                    2. Click "Save Alignment" to save aligned data WITH signals<br>
                    3. Then return here to map signals<br><br>
                    <b>Note:</b> We cannot mix aligned points with raw query signals - this would break the alignment workflow.
                    </span>
                    """
                    status_display.value = "<b>Status:</b> <span style='color: red;'>Error: Aligned data missing signals</span>"
                    progress_bar.value = 0
                    _loading_in_progress = False
                    return
                else:
                    # For raw query data, this is expected if no signals in query results
                    error_display.value = "<span style='color: orange;'>⚠️ No signal data found for selected source. Using sample data.</span>"
                    # Fallback to sample data
                    bbox_min_arr = np.array(bbox_min)
                    bbox_max_arr = np.array(bbox_max)
                    current_points, current_signals = generate_sample_data(
                        n_points=1000,
                        bbox_min=tuple(bbox_min_arr),
                        bbox_max=tuple(bbox_max_arr)
                    )
                    progress_bar.value = 90
                    sample_data_generated = True
                    
                    # Update data info
                    signal_names = list(current_signals.keys())
                    signal_names_display = ', '.join(signal_names[:5])
                    if len(signal_names) > 5:
                        signal_names_display += f", ... and {len(signal_names) - 5} more"
                    
                    data_info.value = f"""
                    <p><b>Model:</b> {loaded_grid_data.get('model_name', 'Unknown')}</p>
                    <p><b>Grid:</b> {loaded_grid_data.get('grid_name', 'Unknown')}</p>
                    <p><b>Data Source:</b> Sample Data (no signals found in query)</p>
                    <p><b>Points:</b> {len(current_points):,}</p>
                    <p><b>Signals ({len(signal_names)}):</b> {signal_names_display}</p>
                    <p><b>Grid BBox:</b> {bbox_min} to {bbox_max}</p>
                    <p><b>Grid Resolution:</b> {resolution} mm</p>
                    """
                    
                    # Update signal selector
                    available_signals = list(current_signals.keys())
                    if available_signals:
                        signal_selector.options = [(s.capitalize(), s) for s in available_signals]
                        signal_selector.value = available_signals[0]
                    
                    progress_bar.value = 100
                    status_display.value = "<b>Status:</b> <span style='color: orange;'>⚠️ Using sample data (no signals in query results)</span>"
                    error_display.value = f"<span style='color: orange;'>⚠️ Loaded {len(current_points):,} points with {len(current_signals)} sample signal(s)</span>"
                    _loading_in_progress = False
                    return
        
        # Only proceed if we have signals (already handled the no-signals case above)
        if not current_signals:
            # This should not happen here if the check above worked, but just in case
            _loading_in_progress = False
            return
        
        progress_bar.value = 90
        sample_data_generated = True
        
        # Update data info
        signal_names = list(current_signals.keys())
        signal_names_display = ', '.join(signal_names[:5])
        if len(signal_names) > 5:
            signal_names_display += f", ... and {len(signal_names) - 5} more"
        
        # Show alignment info if using aligned data
        alignment_info = ""
        if aligned_data_loaded and alignment_id:
            alignment_info = f"<p><b>✅ Using Aligned Data</b> (Alignment ID: {alignment_id[:8]}...)</p>"
        elif created_from_alignment and not aligned_data_loaded:
            alignment_info = f"<p><b>⚠️ Grid from alignment</b> but aligned data could not be loaded</p>"
        
        data_info.value = f"""
        <p><b>Model:</b> {loaded_grid_data.get('model_name', 'Unknown')}</p>
        <p><b>Grid:</b> {loaded_grid_data.get('grid_name', 'Unknown')}</p>
        {alignment_info}
        <p><b>Data Source:</b> {signal_source}</p>
        <p><b>Points:</b> {len(current_points):,}</p>
        <p><b>Signals ({len(signal_names)}):</b> {signal_names_display}</p>
        <p><b>Grid BBox:</b> {bbox_min} to {bbox_max}</p>
        <p><b>Grid Resolution:</b> {resolution} mm</p>
        """
        
        # Update signal selector with available signals
        available_signals = list(current_signals.keys())
        if available_signals:
            signal_selector.options = [(s.capitalize(), s) for s in available_signals]
            signal_selector.value = available_signals[0]
        
        progress_bar.value = 100
        if aligned_data_loaded:
            status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Aligned data loaded successfully</span>"
            error_display.value = f"<span style='color: green;'>✅ Loaded {len(current_points):,} aligned points with {len(current_signals)} signal(s)</span>"
        else:
            status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Data loaded successfully</span>"
            error_display.value = f"<span style='color: green;'>✅ Loaded {len(current_points):,} points with {len(current_signals)} signal(s)</span>"
        
    except Exception as e:
        error_display.value = f"<span style='color: red;'>❌ Error loading data: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error loading data</span>"
        progress_bar.value = 0
        import traceback
        traceback.print_exc()
    finally:
        _loading_in_progress = False

# ============================================
# Mapping Functions
# ============================================

def generate_sample(button):
    """Generate sample point cloud data."""
    global current_points, current_signals, sample_data_generated
    
    status_display.value = "<b>Status:</b> Generating sample data..."
    progress_bar.value = 0
    
    try:
        # Generate data
        bbox_min = (-50.0, -50.0, 0.0)
        bbox_max = (50.0, 50.0, 100.0)
        current_points, current_signals = generate_sample_data(n_points=1000, bbox_min=bbox_min, bbox_max=bbox_max)
        
        sample_data_generated = True
        progress_bar.value = 100
        
        # Update data info
        data_info.value = f"""
        <p><b>Points:</b> {len(current_points):,}</p>
        <p><b>Signals:</b> {', '.join(current_signals.keys())}</p>
        <p><b>BBox:</b> {bbox_min} to {bbox_max}</p>
        """
        
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Sample data generated</span>"
        
        # Update visualization
        update_visualization()
        
    except Exception as e:
        error_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error generating data</span>"
        progress_bar.value = 0

def map_signals(button):
    """Map ALL signals to voxel grid using selected method."""
    global current_points, current_signals, current_grid, mapped_grid, current_model_id
    spark_session_created = None  # Track if we create a Spark session
    
    # Check what's missing and provide helpful error message
    if not sample_data_generated:
        if data_source_mode.value == 'mongodb':
            error_display.value = "<span style='color: red;'>⚠️ Please click 'Load from MongoDB' button first to load data</span>"
        else:
            error_display.value = "<span style='color: red;'>⚠️ Please click 'Generate Sample Data' button first</span>"
        return
    
    if current_points is None:
        error_display.value = "<span style='color: red;'>⚠️ No point data loaded. Please load data first.</span>"
        return
    
    if current_grid is None:
        error_display.value = "<span style='color: red;'>⚠️ No grid loaded. Please load data from MongoDB first.</span>"
        return
    
    if not current_signals:
        error_display.value = "<span style='color: red;'>⚠️ No signals available to map</span>"
        return
    
    status_display.value = "<b>Status:</b> Mapping all signals..."
    progress_bar.value = 0
    error_display.value = ""
    
    try:
        # Use existing grid if loaded, otherwise create new one from points
        if current_grid is None:
            # Create voxel grid from point data bounds
            bbox_min = tuple(current_points.min(axis=0))
            bbox_max = tuple(current_points.max(axis=0))
            resolution = grid_resolution.value
            
            if VOXEL_AVAILABLE:
                current_grid = VoxelGrid(
                    bbox_min=bbox_min,
                    bbox_max=bbox_max,
                    resolution=resolution,
                    aggregation='mean'
                )
            else:
                # Demo grid
                class DemoGrid:
                    def __init__(self, bbox_min, bbox_max, resolution):
                        self.bbox_min = np.array(bbox_min)
                        self.bbox_max = np.array(bbox_max)
                        self.resolution = resolution
                        self.size = self.bbox_max - self.bbox_min
                        self.dims = np.ceil(self.size / resolution).astype(int)
                        self.dims = np.maximum(self.dims, [1, 1, 1])
                        self.voxels = {}
                        self.available_signals = set()
                current_grid = DemoGrid(bbox_min, bbox_max, resolution)
        else:
            # Grid already loaded from MongoDB, use it as-is
            pass
        
        progress_bar.value = 20
        
        # Map ALL signals, not just one
        signals_to_map = current_signals.copy()
        n_signals = len(signals_to_map)
        
        status_display.value = f"<b>Status:</b> Mapping {n_signals} signal(s)..."
        progress_bar.value = 30
        
        # Perform mapping based on method
        method = method_selector.value
        
        # Update status with method info
        status_display.value = f"<b>Status:</b> Mapping {n_signals} signal(s) using {method} method..."
        progress_bar.value = 40
        
        if SIGNAL_MAPPING_AVAILABLE and VOXEL_AVAILABLE:
            # Real mapping - map all signals at once
            import time
            start_time = time.time()
            
            try:
                if method == 'nearest':
                    status_display.value = f"<b>Status:</b> Running nearest neighbor interpolation..."
                    progress_bar.value = 50
                    mapped_grid = interpolate_to_voxels(
                        current_points, signals_to_map, current_grid,
                        method='nearest'
                    )
                elif method == 'linear':
                    status_display.value = f"<b>Status:</b> Running linear interpolation (k={linear_k_neighbors.value})..."
                    progress_bar.value = 50
                    mapped_grid = interpolate_to_voxels(
                        current_points, signals_to_map, current_grid,
                        method='linear',
                        k_neighbors=linear_k_neighbors.value,
                        radius=linear_radius.value if linear_use_radius.value else None
                    )
                elif method == 'idw':
                    status_display.value = f"<b>Status:</b> Running IDW interpolation (power={idw_power.value})..."
                    progress_bar.value = 50
                    mapped_grid = interpolate_to_voxels(
                        current_points, signals_to_map, current_grid,
                        method='idw',
                        power=idw_power.value,
                        k_neighbors=idw_k_neighbors.value,
                        radius=idw_radius.value if idw_use_radius.value else None
                    )
                elif method == 'gaussian_kde':
                    status_display.value = f"<b>Status:</b> Running Gaussian KDE interpolation (bandwidth={kde_bandwidth.value})..."
                    progress_bar.value = 50
                    mapped_grid = interpolate_to_voxels(
                        current_points, signals_to_map, current_grid,
                        method='gaussian_kde',
                        bandwidth=kde_bandwidth.value,
                        adaptive=kde_adaptive.value
                    )
                
                elapsed_time = time.time() - start_time
                progress_bar.value = 90
                status_display.value = f"<b>Status:</b> Mapping completed in {elapsed_time:.2f} seconds"
                
            except Exception as e:
                elapsed_time = time.time() - start_time
                raise Exception(f"Mapping failed after {elapsed_time:.2f}s: {str(e)}")
        else:
            # Demo mapping - create synthetic mapped data
            mapped_grid = current_grid
            # Add available_signals to demo grid
            if hasattr(mapped_grid, 'available_signals'):
                mapped_grid.available_signals = set(signals_to_map.keys())
            # In real implementation, this would contain interpolated values
        
        progress_bar.value = 80
        
        # Update signal selector with mapped signals
        if hasattr(mapped_grid, 'available_signals') and mapped_grid.available_signals:
            available_signals = sorted(list(mapped_grid.available_signals))
            signal_selector.options = [(s.replace('_', ' ').title(), s) for s in available_signals]
            signal_selector.value = available_signals[0]
        elif signals_to_map:
            # Fallback to original signals
            available_signals = sorted(list(signals_to_map.keys()))
            signal_selector.options = [(s.replace('_', ' ').title(), s) for s in available_signals]
            signal_selector.value = available_signals[0]
        
        # Update displays
        update_results_display()
        update_visualization()
        
        # Show save button
        save_mapped_button.layout.display = 'flex'
        
        progress_bar.value = 100
        status_display.value = f"<b>Status:</b> <span style='color: green;'>✅ {n_signals} signal(s) mapped successfully</span>"
        error_display.value = f"<span style='color: green;'>✅ Mapped {n_signals} signal(s) to grid</span>"
        
    except Exception as e:
        error_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error mapping signals</span>"
        progress_bar.value = 0
    finally:
        # Cleanup: Close Spark session if one was created during mapping
        # Note: Currently Spark is not used in the notebook, but this ensures cleanup if added later
        if SPARK_AVAILABLE:
            try:
                from pyspark.sql import SparkSession
                # Get the active Spark session if one exists
                spark = SparkSession.getActiveSession()
                if spark is not None:
                    # Only stop if we're in a notebook environment and session was auto-created
                    # In production, sessions are typically managed externally
                    # For now, we'll leave the session open for potential reuse
                    # Uncomment the line below if you want to close sessions after each mapping:
                    # spark.stop()
                    pass
            except Exception:
                pass  # Ignore errors during cleanup

def update_results_display():
    """Update results and metrics displays."""
    global mapped_grid, current_grid
    
    if mapped_grid is None:
        return
    
    # Results
    if hasattr(mapped_grid, 'dims'):
        dims = mapped_grid.dims
        total_voxels = int(np.prod(dims))
        filled_voxels = len(mapped_grid.voxels) if hasattr(mapped_grid, 'voxels') else 0
        
        results_html = f"""
        <p><b>Grid Dimensions:</b> {dims[0]} × {dims[1]} × {dims[2]}</p>
        <p><b>Total Voxels:</b> {total_voxels:,}</p>
        <p><b>Filled Voxels:</b> {filled_voxels:,}</p>
        <p><b>Coverage:</b> {100 * filled_voxels / total_voxels if total_voxels > 0 else 0:.1f}%</p>
        """
    else:
        results_html = "<p>Results not available</p>"
    
    results_display.value = results_html
    
    # Metrics
    method = method_selector.value
    # Get available signals count
    n_signals = 0
    if hasattr(mapped_grid, 'available_signals'):
        n_signals = len(mapped_grid.available_signals)
    elif hasattr(mapped_grid, 'voxels'):
        # Try to infer from voxels structure
        n_signals = 1  # Default
    
    metrics_html = f"""
    <p><b>Method:</b> {method}</p>
    <p><b>Resolution:</b> {grid_resolution.value} mm</p>
    <p><b>Signals Mapped:</b> {n_signals}</p>
    <p><b>Current Signal:</b> {signal_selector.value}</p>
    """
    
    if method == 'linear':
        metrics_html += f"<p><b>K Neighbors:</b> {linear_k_neighbors.value}</p>"
    elif method == 'idw':
        metrics_html += f"<p><b>Power:</b> {idw_power.value}</p>"
        metrics_html += f"<p><b>K Neighbors:</b> {idw_k_neighbors.value}</p>"
    elif method == 'gaussian_kde':
        metrics_html += f"<p><b>Bandwidth:</b> {kde_bandwidth.value}</p>"
    
    metrics_display.value = metrics_html

def update_visualization():
    """Update visualization display."""
    global current_points, current_signals, mapped_grid
    
    with viz_output:
        clear_output(wait=True)
        
        # If we have a mapped grid, visualize from it
        if mapped_grid is not None:
            signal_name = signal_selector.value
            
            # Check if signal exists in mapped grid
            if hasattr(mapped_grid, 'available_signals') and signal_name in mapped_grid.available_signals:
                # Get signal values from mapped grid
                # For demo, we'll show a placeholder
                display(HTML(f"<h4>Mapped Grid Visualization: {signal_name}</h4>"))
                display(HTML("<p>Grid visualization will be available in future versions.</p>"))
                display(HTML(f"<p><b>Available signals:</b> {', '.join(sorted(mapped_grid.available_signals))}</p>"))
                return
        
        # Fallback to point cloud visualization
        if not sample_data_generated or current_points is None:
            display(HTML("<p>Load data and map signals to see visualization</p>"))
            return
        
        # Create 2D slice visualization
        signal_name = signal_selector.value
        if signal_name not in current_signals:
            display(HTML(f"<p>Signal '{signal_name}' not available</p>"))
            return
        
        signal_values = current_signals[signal_name]
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # Left: Point cloud scatter
        ax1 = axes[0]
        scatter = ax1.scatter(
            current_points[:, 0],
            current_points[:, 1],
            c=signal_values,
            cmap='viridis',
            s=10,
            alpha=0.6
        )
        ax1.set_xlabel('X (mm)')
        ax1.set_ylabel('Y (mm)')
        ax1.set_title(f'Point Cloud: {signal_name}')
        plt.colorbar(scatter, ax=ax1, label=signal_name)
        
        # Right: Grid visualization (if mapped)
        ax2 = axes[1]
        if mapped_grid is not None and hasattr(mapped_grid, 'dims'):
            # Create a simple 2D representation
            dims = mapped_grid.dims
            data = np.zeros((dims[1], dims[0]))
            # In real implementation, would extract signal values from grid
            im = ax2.imshow(data, cmap='viridis', origin='lower', aspect='auto')
            ax2.set_title(f'Mapped Grid: {signal_name}')
            ax2.set_xlabel('X (voxels)')
            ax2.set_ylabel('Y (voxels)')
            plt.colorbar(im, ax=ax2, label=signal_name)
        else:
            ax2.text(0.5, 0.5, 'Map signals to see grid visualization', 
                    ha='center', va='center', transform=ax2.transAxes)
            ax2.set_title('Grid Visualization')
        
        plt.tight_layout()
        plt.show()

def save_mapped_grid(button):
    """Save the mapped grid with all signals to MongoDB."""
    global mapped_grid, current_model_id, current_grid_id, voxel_storage
    
    if mapped_grid is None:
        error_display.value = "<span style='color: red;'>⚠️ No mapped grid to save. Please map signals first.</span>"
        return
    
    if not current_model_id:
        error_display.value = "<span style='color: red;'>⚠️ No model selected. Cannot save grid.</span>"
        return
    
    if not voxel_storage:
        error_display.value = "<span style='color: red;'>⚠️ VoxelGridStorage not available.</span>"
        return
    
    status_display.value = "<b>Status:</b> Saving mapped grid..."
    progress_bar.value = 0
    error_display.value = ""
    
    try:
        # Generate grid name
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        method = method_selector.value
        grid_name = f"mapped_{method}_{timestamp}"
        
        # Get model name if available
        model_name = None
        if stl_client:
            try:
                model_info = stl_client.get_model(current_model_id)
                if model_info:
                    model_name = model_info.get('model_name') or model_info.get('filename', 'Unknown')
            except:
                pass
        
        # Configuration metadata
        config_metadata = {
            'mapping_method': method,
            'grid_resolution': grid_resolution.value,
            'mapped_signals': sorted(list(mapped_grid.available_signals)) if hasattr(mapped_grid, 'available_signals') else [],
        }
        
        if method == 'linear':
            config_metadata['linear_k_neighbors'] = linear_k_neighbors.value
            config_metadata['linear_radius'] = linear_radius.value if linear_use_radius.value else None
        elif method == 'idw':
            config_metadata['idw_power'] = idw_power.value
            config_metadata['idw_k_neighbors'] = idw_k_neighbors.value
            config_metadata['idw_radius'] = idw_radius.value if idw_use_radius.value else None
        elif method == 'gaussian_kde':
            config_metadata['kde_bandwidth'] = kde_bandwidth.value
            config_metadata['kde_adaptive'] = kde_adaptive.value
        
        progress_bar.value = 50
        
        # Save grid
        saved_grid_id = voxel_storage.save_voxel_grid(
            model_id=current_model_id,
            grid_name=grid_name,
            voxel_grid=mapped_grid,
            description=f"Mapped grid with {len(config_metadata['mapped_signals'])} signal(s) using {method} method",
            model_name=model_name,
            configuration_metadata=config_metadata
        )
        
        current_grid_id = saved_grid_id
        
        progress_bar.value = 100
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Mapped grid saved successfully</span>"
        error_display.value = f"<span style='color: green;'>✅ Saved grid: {grid_name} (ID: {saved_grid_id[:8]}...)</span>"
        
    except Exception as e:
        error_display.value = f"<span style='color: red;'>❌ Error saving grid: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error saving grid</span>"
        progress_bar.value = 0

def load_mapped_grid(button):
    """Load a previously mapped grid from MongoDB."""
    global mapped_grid, current_model_id, current_grid_id, loaded_grid_data, voxel_storage
    
    if not voxel_storage or not current_model_id:
        error_display.value = "<span style='color: red;'>⚠️ Please select a model first</span>"
        return
    
    status_display.value = "<b>Status:</b> Loading mapped grid..."
    progress_bar.value = 0
    error_display.value = ""
    
    try:
        # List available grids for the model
        grids = voxel_storage.list_grids(model_id=current_model_id)
        
        if not grids:
            error_display.value = "<span style='color: orange;'>⚠️ No grids found for this model</span>"
            return
        
        # Filter for mapped grids (those with signals)
        mapped_grids = [g for g in grids if g.get('available_signals')]
        
        if not mapped_grids:
            error_display.value = "<span style='color: orange;'>⚠️ No mapped grids found (grids with signals)</span>"
            return
        
        # Load the most recent mapped grid (or could use grid_dropdown.value)
        # Note: list_grids() returns grids with 'grid_id', not '_id'
        grid_to_load = grid_dropdown.value if grid_dropdown.value else mapped_grids[0].get('grid_id')
        
        progress_bar.value = 30
        
        # Load grid
        loaded_grid_data = voxel_storage.load_voxel_grid(grid_to_load)
        
        if not loaded_grid_data:
            error_display.value = f"<span style='color: red;'>❌ Failed to load grid</span>"
            return
        
        progress_bar.value = 60
        
        # Reconstruct grid from loaded data
        metadata = loaded_grid_data.get('metadata', {})
        bbox_min = tuple(metadata.get('bbox_min', [-50, -50, 0]))
        bbox_max = tuple(metadata.get('bbox_max', [50, 50, 100]))
        resolution = metadata.get('resolution', 2.0)
        
        if VOXEL_AVAILABLE:
            mapped_grid = VoxelGrid(
                bbox_min=bbox_min,
                bbox_max=bbox_max,
                resolution=resolution,
                aggregation=metadata.get('aggregation', 'mean')
            )
            # Load signal arrays into grid
            signal_arrays = loaded_grid_data.get('signal_arrays', {})
            if hasattr(mapped_grid, 'available_signals'):
                mapped_grid.available_signals = set(loaded_grid_data.get('available_signals', []))
        else:
            # Demo grid
            class DemoGrid:
                def __init__(self, bbox_min, bbox_max, resolution):
                    self.bbox_min = np.array(bbox_min)
                    self.bbox_max = np.array(bbox_max)
                    self.resolution = resolution
                    self.size = self.bbox_max - self.bbox_min
                    self.dims = np.ceil(self.size / resolution).astype(int)
                    self.dims = np.maximum(self.dims, [1, 1, 1])
                    self.voxels = {}
                    self.available_signals = set(loaded_grid_data.get('available_signals', []))
            mapped_grid = DemoGrid(bbox_min, bbox_max, resolution)
        
        current_grid_id = grid_to_load
        
        progress_bar.value = 80
        
        # Update signal selector with available signals
        available_signals = loaded_grid_data.get('available_signals', [])
        if available_signals:
            signal_selector.options = [(s.replace('_', ' ').title(), s) for s in sorted(available_signals)]
            signal_selector.value = sorted(available_signals)[0]
        
        # Update displays
        update_results_display()
        update_visualization()
        
        progress_bar.value = 100
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Mapped grid loaded successfully</span>"
        error_display.value = f"<span style='color: green;'>✅ Loaded grid: {loaded_grid_data.get('grid_name', 'Unknown')} with {len(available_signals)} signal(s)</span>"
        
    except Exception as e:
        error_display.value = f"<span style='color: red;'>❌ Error loading grid: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error loading grid</span>"
        progress_bar.value = 0

# Connect events
load_data_button.on_click(load_from_mongodb)
generate_data_button.on_click(generate_sample_data)
map_button.on_click(map_signals)
save_mapped_button.on_click(save_mapped_grid)
load_mapped_button.on_click(load_mapped_grid)
signal_selector.observe(lambda x: update_visualization(), names='value')
slice_axis.observe(lambda x: update_visualization(), names='value')
slice_position.observe(lambda x: update_visualization(), names='value')

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to map signals to voxel grids using various interpolation methods.

### Key Takeaways

1. **Interpolation Methods**: Nearest Neighbor, Linear, IDW, and Gaussian KDE each have different characteristics
2. **Parameter Tuning**: Method parameters significantly affect mapping quality and performance
3. **Method Selection**: Choose method based on data characteristics and requirements
4. **Visualization**: Compare point cloud data with mapped grid results

### Next Steps

Proceed to:
- **04_Temporal_and_Spatial_Alignment.ipynb** - Learn synchronization and alignment
- **05_Data_Correction_and_Processing.ipynb** - Learn geometric correction and signal processing

### Related Resources

- Signal Mapping Module Documentation: `../docs/AM_QADF/05-modules/signal-mapping.md`
- API Reference: `../docs/AM_QADF/06-api-reference/signal-mapping-api.md`
- Examples: `../examples/`
